In [1]:
"""
Original kernel:non-blending lightGBM model LB: 0.977:https://www.kaggle.com/bk0000/non-blending-lightgbm-model-lb-0-977?scriptVersionId=3224614
V0 Modified by Andy:Kaggle-runnable version of Baris Kanber's LightGBM:https://www.kaggle.com/aharless/kaggle-runnable-version-of-baris-kanber-s-lightgbm/comments
A non-blending lightGBM model that incorporates portions and ideas from various public kernels.

经验教训：
1.70%的时间在做特征工程，使用lag feature滞后特征。
2.找队友，尽量使用云而非自己电脑的windows。
3.最后时间一定要随时刷新kernels。
https://www.kaggle.com/c/talkingdata-adtracking-fraud-detection/discussion/56283
https://www.kaggle.com/c/talkingdata-adtracking-fraud-detection/discussion/56262

Modified by Bai:-----------------------------------------------------------------------------------------------------
References:
- Python学习笔记——可变对象和不可变对象:https://blog.csdn.net/taohuaxinmu123/article/details/39008281
- python id()函数, id()函数用于获取对象的内存地址。:https://www.cnblogs.com/dplearning/p/5998112.html
- def concatenate_block_managers(Source Code):https://github.com/pandas-dev/pandas/blob/v0.22.0/pandas/core/internals.py
* del:Reference count-1, gc.collect():manully garbage clean
- Frequently using small parameters have large reference count:https://segmentfault.com/q/1010000000509607
- pandas的4种引用与3种复制，是否copy只取决于采用了切片还是花式索引。：https://blog.csdn.net/qtlyx/article/details/70500145
- Python内存池管理与缓冲池设计:https://blog.csdn.net/zhzhl202/article/details/7547445
- python Pandas DataFrame copy(deep=False) vs copy(deep=True) vs '=':https://stackoverflow.com/questions/46327494/python-pandas-dataframe-copydeep-false-vs-copydeep-true-vs#
* Deep copy creates new id's of every object it contains while normal copy only copies the elements from the parent and creates a new id for a variable to which it is copied to.
- 机器不学习：一文看懂机器学习时代神器—LightGBM:http://www.360doc.com/content/17/1231/23/40769523_718019029.shtml
- 比XGBOOST更快--LightGBM介绍:https://www.jianshu.com/p/48e82dbb142b
* xgboost is a little bit more accurate than lightgbm, but use 8*RAM than lightgbm
- 机器学习中，有哪些特征选择的工程方法？：https://www.zhihu.com/question/28641663/answer/107680749
* Feature selection exhaustivity is O(2^n), usally use greedy method(forward,backwrad) to find the second-best solution
- Pearson相关系数是用来衡量两个数据集合是否在一条线上面，它用来衡量定距变量间的线性关系。相关系数是研究变量之间线性相关程度的量。https://baike.baidu.com/item/Pearson%E7%9B%B8%E5%85%B3%E7%B3%BB%E6%95%B0/6243913
- pandas.DataFrame.astype:http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.astype.html
* 使用单独的ip作为特征没有意义,与其他特征结合使用。
* Order of features do affect the performance of LGB.
- DataFrame过滤数据时出现"The truth value of a Series is ambiguous"错误及其解决办法:https://blog.csdn.net/l460133921/article/details/80101249
- Pandas速查手册中文版:https://blog.csdn.net/qq_33399185/article/details/60872853
- https://media.readthedocs.org/pdf/lightgbm/latest/lightgbm.pdf
* The experiment on Expo data shows about 8x speed-up compared with one-hot coding.
* LightGBM offers good accuracy when using native categorical features instead of one-hot coding.
- fltsm特征选择神经网络，自带特征变换与选择功能，gcforest
- https://github.com/Microsoft/LightGBM/issues/695
- https://sites.google.com/view/lauraepp/parameters

根本目标：在test_df（10th:4,5,9,10,13,14 +8h）的得分尽量高。
假设(已验证)：这四天中每一天的总体分布相似，故将所有数据合在一起生成新的特征逼近总体分布。
重要：避免过拟合LB，相信本地valid score,与测试集有些许不同

Possible improvement:
* Use lightgbm continue training to train full dataset?'init_model'：直接使用所有数据
* next_click?
* combine the advantages of former kernels?
- tune the parameters?
* consider using blending to csv?: Overfit the LB?
- 【use base model + regressor to blend?】
* Add new features on entire dataset one by one?
- feature selection
* For my own validation (applying full predictions from script, shifted back by 1 day, to test set analogue)
- learning rate epsiolon decay
* convert dataset 'float32' to 'float64'
* convert 'hour' to 3 categories
* Add test_supplyment to generate the features：符合假设，正在使用
* 99.75:0.25, so scale_pos_weight to 150-250?
* 4.26之前历史记录
* find more paper anout how we can define fraud click manully
* Use PCA to generate the feature.
* Add new features!!!!!
- 【Use full train.csv to train with validation最后再尝试。】
* consider use only 9th:4,5,9,10,13,14 +8h to validate, all data train: overfitting
* delete 9th 4h validate??
* Fix 18 testdata mapping click_id
* skew and var
* Cumcount features becomes bigger and bigger, I tried do cumcount group by day and got a better score

V0: * Use functions to optimize the coding structure, useful to debug
    * Add many new group by features and 'nextClick'
    3372.4 seconds,[127]	train's auc: 0.987305	valid's auc: 0.972478, LB:0.9761, nchunk=25000000
V1: - Add all 'merge' function 'copy=False'，USEFUL for at least computational speed and prevent reach the RAM limit
    Ran 2923 seconds, [129]	train's auc: 0.987555	valid's auc: 0.972648, LB:0.9764
V2: - Add all 'astype' function 'copy=False'
    * Use default seed: 'data_random_seed','bagging_seed','feature_fraction_seed':1,2,3
    - Add lgb.plot_importance() for both 'split' and 'gain'
    - Reorganize the lgb training function
    * 2'nextClick' features contains large randomness but EXTREMELY useful
    * delete 2'nextClick' features, result same to the fork, [231]	train's auc: 0.984031	valid's auc: 0.96488, LB:0.9661
    * then delete lowest score 'ip_app_channel_mean_hour' feature, [124]train's auc: 0.981878	valid's auc: 0.965349, LB:0.9662
V3: * 'nthread':for the best speed, set this to the number of real CPU cores, not the number of threads:http://lightgbm.readthedocs.io/en/latest/Parameters.html
    - delete all the gc.collect() in the feature function
    * Different 'nthread' lead to different result. It is fixed to 4 after test.
    * Same result with along same code can have large gap in time, maybe due to the server disk, so time IS NOT fixed
    - Train on local computer, using GPU
    - Using new method to generate feature 'nextClick' rather than hash
    kaggle cpu:[151]	train's auc: 0.988041	valid's auc: 0.973151,SAME to the fork, LB:0.9770, 3425.9s
    local cpu:[64]	train's auc: 0.985661	valid's auc: 0.972455,SAME to the fork, LB:0.9761, 1625s
    local gpu:[105]	train's auc: 0.987103	valid's auc: 0.97292，2235s
    local gpu double precesion:[64]	train's auc: 0.985661	valid's auc: 0.972455, 1857s, SAME to local cpu
V4: - change nchunk from 25000000 to 75000000
    CPU:[326]	train's auc: 0.985568	valid's auc: 0.990381，LB:0.9794, use 30G swap, almost 5hour
    - change nchunk from 75000000 to 140000000
    - delete 'day'.'day'只是用来切块每一天的某小时(区分不同天)，本身不用作分类
    - delete 'ip_tchan_count','ip_app_channel_mean_hour','ip_app_os_var','X7' temporarily for speed
    - [162]train's auc: 0.984942	valid's auc: 0.99025, 19709s, LB:0.9791
V5: - add 'ip_tchan_count','ip_app_channel_mean_hour','ip_app_os_var','X7' back(IMPORTANT)
    - change nchunk from 140000000 to 150000000
    [327]	train's auc: 0.985816	valid's auc: 0.990797
    - add 'day' for test
    - early_stopping_rounds=30 to 50
    [650]	train's auc: 0.986541	valid's auc: 0.99073, 16G RAM+62G swap, 28562s, LB:0.9796
V6: - delete 'day'
    - change nchunk from 150000000 to all
    - convert train/valid/test to 'float32' before beginning training to prevent memory spike
    [309]	train's auc: 0.985642	valid's auc: 0.990719, 16G RAM+40G swap, 23376s, LB:0.9798
    [15766.776413679123]: model training time
V7: * Use GPU(GPU0):LightGBMError: b'Invalid Buffer Size', no GPU from now on
    - Add all time 8hour to combine three complete days
    * 0-482 on 6th after +8h:ingnore, 184903443-184903889 on 10th after +8h
    - Save train/valid/test csv to disk for quickly reading
    - change val_df from 2.5M to last 40M(40000000) because test_df begins at 4h to 15h too(influnce early stop)
    * valid should only include 9th(4,5,9,10,13,14 +8h), train use only 7th, 8th
    [525]	train's auc: 0.986902	valid's auc: 0.983006, 23155s, LB:0.9795
    [21972.033691883087]: model training time
V8: - Use 9th(4,5,9,10,13,14 +8h) as validation, simply drop other hours on 9th NOT use to train
    - csv take too much space, use pickle instead
    - delete xgtrain in valid_sets for training speed
    - add new feature: 'A0', 'A1', 'A2'
    [630]	valid's auc: 【0.982519】, LB:0.9792, 122070801 training data, 20895641 validation data
V9: - Use 9th(4,5,9,10,13,14 +8h) as validation, other hours use for training
    - 'uint32' to 'float32' lead to float rounding, fix it for 'click_id'
    [544]	valid's auc: 【0.983071】,[5830.981881141663]: model training time
    164008249 training data,20895641 valid, LB:0.9796
V10: * 'scale_pos_weight': 200 to 400, LB:0.9793
V11: - Add test_supplyment to generate features together
     * 'click_id'会乱序，应使用原文件的'click_id'而非index
     [512]	valid's auc: 【0.98313】, Preparing data time: 17311, LB:0.8159 -> 【0.9810】
     * 'scale_pos_weight':300, [507]	valid's auc: 0.983033, [5520.847676038742]: model training time
     * learning_rate 0.2 to 0.1
     [986]	valid's auc: 0.983183, [9227.210345506668]: model training time
     * 'scale_pos_weight':250, [706]	valid's auc: 0.983098
     - Fix 'click_id' mapping
V12: * learning_rate 0.2 to 0.08
     [1593]	valid's auc: 【0.98322】, 【LB:0.9811】, 72rd
V13: - delete 'X7'双倒数第一
     - set learning_rate to 0.2 for speed temporarily
     [788]	valid's auc: 0.983164, LB:0.9810
V14: - add 4 nextClick features
     [481]	valid's auc: 0.983143，LB：0.9809
V15: - delete 'nextClick3'
     [512]	valid's auc: 【0.98322, LB:0.9811】
V16: - add 5 prevClick features
     [803]	valid's auc: 0.983308,内存已提交76G,11221s, LB:0.9807
V17: - delete 'prevClick'
     [394]	valid's auc: 0.983209, LB:0.9807
V18: - delete 'prevClick3','prevClick5'
     [423]	valid's auc: 0.983272, LB:0.9807
V19: - delete 'prevClick2'
     [404]	valid's auc: 0.983068，LB:0.9809
V20: - delete all 'prevClick' useless,相似的特征加入会削弱其他特征的影响
     * delete 'A1',add back
     [550]	valid's auc: 0.98312
V21: - add nextClick6,B0-B4, LB:0.9809
V22: - add 'C0','C1'
     - delete 'X6'双倒一，'ip_tchan_count'
     [456]	valid's auc: 0.983322, LB:0.9810
V23: - delete 'B0','A1','ip_app_channel_var_day'
     [508]	valid's auc: 【【0.983372, LB:0.9812】】
V24: - set learning_rate from 0.2 to 0.05, num_boost_round=3000,early_stopping_rounds=80
     - transfer Do to main function
     [2018]	valid's auc: 【0.983498, LB:0.9813】
"""
import pandas as pd
import time
import numpy as np
from sklearn.model_selection import train_test_split
import lightgbm as lgb
import gc
import matplotlib.pyplot as plt
import os
from pandas.tseries.offsets import DateOffset
# from sklearn.metrics import roc_auc_score

def do_count( df, group_cols, agg_name, agg_type='uint32', show_max=False):
    print( "do_count to ", group_cols , '...' )
    gp = df[group_cols].groupby(group_cols).size().rename(agg_name).to_frame().reset_index()
#     if debug:
#         print(df[group_cols])
# #         print(df[group_cols][group_cols]) # same to the former
#         print(df[group_cols][group_cols].groupby(group_cols).size())
    df = df.merge(gp, on=group_cols, how='left', copy=False)
    del gp
    if show_max:
        print( agg_name + " max value = ", df[agg_name].max())
    df[agg_name] = df[agg_name].astype(agg_type,copy=False)
    return( df )

def do_countuniq( df, group_cols, counted, agg_name, agg_type='uint32', show_max=False):
    print( "Counting unqiue ", counted, " by ", group_cols , '...' )
    # print('the Id of train_df while function before merge: ',id(df)) # the same with train_df
    gp = df[group_cols+[counted]].groupby(group_cols)[counted].nunique().reset_index().rename(columns={counted:agg_name})
    df = df.merge(gp, on=group_cols, how='left', copy=False)
    # print('the Id of train_df while function after merge: ',id(df)) # id changes
    del gp
    if show_max:
        print( agg_name + " max value = ", df[agg_name].max() )
    df[agg_name] = df[agg_name].astype(agg_type,copy=False)
    return( df )
    
def do_cumcount( df, group_cols, counted, agg_name, agg_type='uint32', show_max=False):
    print( "Cumulative count by ", group_cols , '...' )
    gp = df[group_cols+[counted]].groupby(group_cols)[counted].cumcount() # 累加
    df[agg_name]=gp.values
    del gp
    if show_max:
        print( agg_name + " max value = ", df[agg_name].max() )
    df[agg_name] = df[agg_name].astype(agg_type,copy=False)
    return( df )

def do_mean( df, group_cols, counted, agg_name, agg_type='float32', show_max=False):
    print( "Calculating mean of ", counted, " by ", group_cols , '...' )
    gp = df[group_cols+[counted]].groupby(group_cols)[counted].mean().reset_index().rename(columns={counted:agg_name})
    df = df.merge(gp, on=group_cols, how='left', copy=False)
    del gp
    if show_max:
        print( agg_name + " max value = ", df[agg_name].max() )
    df[agg_name] = df[agg_name].astype(agg_type,copy=False)
    return( df )

def do_var( df, group_cols, counted, agg_name, agg_type='float32', show_max=False):
    print( "Calculating variance of ", counted, " by ", group_cols , '...' )
    gp = df[group_cols+[counted]].groupby(group_cols)[counted].var().reset_index().rename(columns={counted:agg_name})
    df = df.merge(gp, on=group_cols, how='left', copy=False)
    del gp
    if show_max:
        print( agg_name + " max value = ", df[agg_name].max() )
    df[agg_name] = df[agg_name].astype(agg_type,copy=False)
    return( df )

def lgb_modelfit_nocv(dtrain, dvalid, predictors, target='target', feval=None, early_stopping_rounds=30, num_boost_round=3000, verbose_eval=10, categorical_features=None,metrics='auc'):
    lgb_params = {
        'boosting_type': 'gbdt',
        'objective': 'binary',
        'metric':'auc',
        'learning_rate': 0.05, # 【consider use 0.2,0.1,0.05,0.02】
        #'is_unbalance': 'true',  #because training data is unbalance (replaced with scale_pos_weight)
        'scale_pos_weight': 200, # because training data is extremely unbalanced
        'num_leaves': 7,  # we should let it be smaller than 2^(max_depth), default=31
        'max_depth': 3,  # -1 means no limit, default=-1
        'min_data_per_leaf': 100,  # alias=min_data_per_leaf , min_data, min_child_samples, default=20
        'max_bin': 100,  # Number of bucketed bin for feature values,default=255,set this small for more GPU speed
        'subsample': 0.7,  # Subsample ratio of the training instance.default=1.0, alias=bagging_fraction
        'subsample_freq': 1,  # k means will perform bagging at every k iteration, <=0 means no enable,alias=bagging_freq,default=0
        'colsample_bytree': 0.9,  # Subsample ratio of columns when constructing each tree.alias:feature_fraction
        'min_child_weight': 0,  # Minimum sum of instance weight(hessian) needed in a child(leaf),default=1e-3,Like min_data_in_leaf, it can be used to deal with over-fitting
        'subsample_for_bin': 200000,  # Number of samples for constructing bin
        'min_split_gain': 0,  # lambda_l1, lambda_l2 and min_gain_to_split to regularization
        'reg_alpha': 0,  # L1 regularization term on weights，【consider use to avoid overfit】
        'reg_lambda': 0,  # L2 regularization term on weights
        'nthread': 4, # should be equal to REAL cores:http://xgboost.readthedocs.io/en/latest/how_to/external_memory.html
        'verbose': 0
        
#         'device': 'gpu',
#         'gpu_platform_id':1
        # gpu_use_dp, default=false,set to true to use double precision math on GPU (default using single precision)
#         'gpu_device_id': 2 #default=-1,default value is -1, means the default device in the selected platform
        # 'random_state':666 [LightGBM] [Warning] Unknown parameter: random_state
        # 'feature_fraction_seed': 666,
        # 'bagging_seed': 666, # alias=bagging_fraction_seed
        # 'data_random_seed': 666 # random seed for data partition in parallel learning (not include feature parallel)
    }
    # lgb_params.update(params) # Python dict.update()

    print("load train_df into lgb.Dataset...")
    # free_raw_data (bool, optional (default=True)) – If True, raw data is freed after constructing inner Dataset.
    xgtrain = lgb.Dataset(dtrain[predictors].values, label=dtrain[target].values,
                          feature_name=predictors,
                          categorical_feature=categorical_features
                          )
    del dtrain
    print("load valid_df into lgb.Dataset...")
    xgvalid = lgb.Dataset(dvalid[predictors].values, label=dvalid[target].values,
                          feature_name=predictors,
                          categorical_feature=categorical_features
                          )
    del dvalid
    gc.collect()

    evals_results = {}
    
    # Warning:basic.py:681: UserWarning: categorical_feature in param dict is overrided.
    # https://github.com/Microsoft/LightGBM/blob/master/python-package/lightgbm/basic.py#L679
    # https://github.com/Microsoft/LightGBM/blob/master/python-package/lightgbm/basic.py#L483
    print('Start doing lgb.train...')
    bst1 = lgb.train(lgb_params, 
                     xgtrain, 
                     valid_sets=[xgvalid], 
                     valid_names=['valid'], 
                     evals_result=evals_results, 
                     num_boost_round=num_boost_round,
                     early_stopping_rounds=early_stopping_rounds,
                     verbose_eval=10, 
                     feval=feval)
    
    del xgtrain, xgvalid
    print("\nModel Report")
    print("bst1.best_iteration: ", bst1.best_iteration)
    print(metrics+":", evals_results['valid'][metrics][bst1.best_iteration-1])
    gc.collect()

    return (bst1,bst1.best_iteration)

# --------------------------------------------------------------------------------------------------------------
# --------------------------------------------------------------------------------------------------------------

# Main function-------------------------------------------------------------------------------------
if __name__ == '__main__':
    print('Beginning...',time.asctime( time.localtime(time.time())))
    script_start_time=time.time()
    
    nrows=184903891 # the first line is columns' name
#     nchunk=150000000
    nchunk=184903891 # entire dataset
    val_size=62833089  # since 9th 0:00:00
    frm=nrows-nchunk
    
    debug=False
#     debug=True
#     read=False # whether to read data from file rather than generate and save data file
    read=True
    if debug:
        print('*** Debug: this is a test run for debugging purposes ***')
        frm=0
        nchunk=100000
        val_size=10000
    
    to=frm+nchunk
    
#     DO(frm,to,24) # Version NO. start from 0
    fileno = 24
    
    if not read:
        dtypes = {
                'ip'            : 'uint32',
                'app'           : 'uint16',
                'device'        : 'uint16',
                'os'            : 'uint16',
                'channel'       : 'uint16',
                'is_attributed' : 'uint8', 
                'click_id'      : 'uint32',
                }

        print('loading train data...',frm,'to',to)
        # usecols:Using this parameter results in much faster parsing time and lower memory usage.
        train_df = pd.read_csv("../input/train.csv", parse_dates=['click_time'], skiprows=range(1,frm), nrows=to-frm, dtype=dtypes, usecols=['ip','app','device','os', 'channel', 'click_time', 'is_attributed'])

        print('loading test data...')
        if debug:
            test_df = pd.read_csv("../input/test_supplement.csv", nrows=100000, parse_dates=['click_time'], dtype=dtypes, usecols=['ip','app','device','os', 'channel', 'click_time', 'click_id'])
        else:
            test_df = pd.read_csv("../input/test_supplement.csv", parse_dates=['click_time'], dtype=dtypes, usecols=['ip','app','device','os', 'channel', 'click_time', 'click_id'])

        print('Combine test_df and train_df...')
        len_train = len(train_df)
        # Append rows of other to the end of this frame, returning a new object.
        train_df=train_df.append(test_df) # Shouldn't process individually,because of lots of count,mean,var variables
        train_df['is_attributed'].fillna(-1,inplace=True)
        train_df['is_attributed'] = train_df['is_attributed'].astype('uint8',copy=False)
        train_df['click_id'].fillna(-1,inplace=True)
        train_df['click_id'] = train_df['click_id'].astype('uint32',copy=False)
        del test_df
        gc.collect()

        print('Extracting hour and day and add 8h...')
    #     print(type(train_df['click_time']),type(pd.to_datetime(train_df.click_time))) # Series
    #     print(train_df['click_time'],pd.to_datetime(train_df.click_time)) # dtype: datetime64[ns]
        # http://pandas.pydata.org/pandas-docs/stable/timeseries.html
    #     print(train_df.click_time.head(500)) # 500 data before 6th 16:00
        train_df.click_time = train_df.click_time + DateOffset(hours=8)
    #     print(train_df.click_time.head(500))
        train_df['hour'] = train_df.click_time.dt.hour.astype('uint8')
        train_df['day'] = train_df.click_time.dt.day.astype('uint8')
        gc.collect()

        # print('the Id of train_df before function: ',id(train_df))
        train_df = do_countuniq( train_df, ['ip'], 'channel', 'X0', 'uint8', show_max=False ); gc.collect()
        # print('the Id of train_df after function: ',id(train_df)) # the same id with 'df' returned
        train_df = do_countuniq( train_df, ['ip', 'day'], 'hour', 'X2', 'uint8', show_max=False ); gc.collect()
        train_df = do_countuniq( train_df, ['ip'], 'app', 'X3', 'uint16', show_max=False ); gc.collect()
        train_df = do_countuniq( train_df, ['ip', 'app'], 'os', 'X4', 'uint8', show_max=False ); gc.collect()
        train_df = do_countuniq( train_df, ['ip'], 'device', 'X5', 'uint16', show_max=False ); gc.collect()
        train_df = do_countuniq( train_df, ['app'], 'channel', 'X6','uint8', show_max=False ); gc.collect()
        train_df = do_countuniq( train_df, ['ip', 'device', 'os'], 'app', 'X8','uint8', show_max=False ); gc.collect()
# drop         train_df = do_cumcount( train_df, ['ip'], 'os', 'X7', show_max=False ); gc.collect()
        train_df = do_cumcount( train_df, ['ip', 'device', 'os'], 'app', 'X1', show_max=False ); gc.collect()
        train_df = do_countuniq( train_df, ['ip', 'device', 'os'], 'channel', 'A0', show_max=False ); gc.collect()
        train_df = do_count( train_df, ['ip', 'app', 'channel'], 'A1', show_max=False ); gc.collect()
        train_df = do_count( train_df, ['ip', 'device', 'os','app'], 'A2', show_max=False ); gc.collect()
        # ip-device-hour?

        train_df = do_count( train_df, ['ip', 'day', 'hour'], 'ip_tcount','uint16',show_max=False ); gc.collect()
    #     train_df = do_count( train_df, ['ip', 'hour'], 'ip_tcount2','uint32',show_max=False ); gc.collect()
        train_df = do_count( train_df, ['ip', 'app'], 'ip_app_count','uint32', show_max=False ); gc.collect()
        train_df = do_count( train_df, ['ip', 'app', 'os'], 'ip_app_os_count', 'uint16', show_max=False ); gc.collect()
        train_df = do_var( train_df, ['ip', 'day', 'channel'], 'hour', 'ip_tchan_count', show_max=False ); gc.collect()
        train_df = do_var( train_df, ['ip', 'app', 'os'], 'hour', 'ip_app_os_var', show_max=False ); gc.collect()
        train_df = do_var( train_df, ['ip', 'app', 'channel'], 'day', 'ip_app_channel_var_day', show_max=False ); gc.collect()
        train_df = do_mean( train_df, ['ip', 'app', 'channel'], 'hour', 'ip_app_channel_mean_hour', show_max=False ); gc.collect()

    #=====================================================================================================
        print('doing nextClick without hash...')

        train_df['click_time'] = (train_df['click_time'].astype(np.int64,copy=False) // 10 ** 9).astype(np.int32,copy=False)
        train_df['nextClick'] = (train_df.groupby(['ip', 'app', 'device', 'os']).click_time.shift(-1) - train_df.click_time).astype(np.float32,copy=False)
        print(train_df['nextClick'].head(50))
        train_df.drop(['click_time','ip','day'], axis=1, inplace=True)
        gc.collect()

    #----------------------------------------------------------------------------------------------------------------
        print("vars and data type: ")
        # Warning:A value is trying to be set on a copy of a slice from a DataFrame: only test_df
        test_df = train_df[len_train:]
        test_df.drop(columns='is_attributed',inplace=True)
        train_df.drop(columns='click_id',inplace=True)
        val_df = train_df[(len_train-val_size):len_train] # Validation set
        train_df = train_df[:(len_train-val_size)]
        gc.collect()
        
        print('Generate real testset from test_supplement...')
        test_df_real = test_df[21290876:27493809]
        test_df_real = pd.concat([test_df_real,test_df[35678696:41791910]], copy=False)
        test_df_real = pd.concat([test_df_real,test_df[48109937:54584259]], copy=False)
        test_df = test_df_real
        test_df.drop(columns='click_id',inplace=True)
        gc.collect()

        print("train size: ", len(train_df))
        print("valid size: ", len(val_df))
        print("test size : ", len(test_df))
        
        print('Converting train/valid into float32...')
        train_df = train_df.astype('float32',copy=False) # categories类型也转换为float32：精度没有问题
        val_df = val_df.astype('float32',copy=False)
        print(train_df.info())
        print(val_df.info())
        print(test_df.info())
        gc.collect()
        
        print('Saving data to disk...')
        train_df.to_pickle('./train_df_V%d.pkl'%fileno)
        val_df.to_pickle('./val_df_V%d.pkl'%fileno)
        test_df.to_pickle('./test_df_V%d.pkl'%fileno)

        print('Preparing data time:',time.time()-script_start_time)
        
    else:
        print('Reading dataset from disk file...')
        read_fileno = 21
        train_df = pd.read_pickle('./train_df_V%d.pkl'%read_fileno)
        train_df.drop(columns=['X6','ip_tchan_count','ip_app_channel_var_day','B0','A1'],inplace=True)
        gc.collect()
        val_df = pd.read_pickle('./val_df_V%d.pkl'%read_fileno)
        val_df.drop(columns=['X6','ip_tchan_count','ip_app_channel_var_day','B0','A1'],inplace=True)
        gc.collect()
#         print(train_df.info())
#         print(val_df.info())
#         print(test_df.info())
        trainadd = pd.read_pickle('train_df_feature4.pkl')
        valadd = pd.read_pickle('val_df_feature4.pkl')
        
        train_df['C0'] = trainadd['C0'].values
        train_df['C1'] = trainadd['C1'].values
        val_df['C0'] = valadd['C0'].values
        val_df['C1'] = valadd['C1'].values

        del trainadd,valadd
        gc.collect()
        print('Loading data time:',time.time()-script_start_time)
    
    predictors=[]
    target = 'is_attributed'
    predictors.extend(['app','device','os', 'channel', 'hour','nextClick',
                  'ip_tcount', 'ip_app_count',
                  'ip_app_os_count','ip_app_channel_mean_hour','ip_app_os_var',
                  'X0', 'X1', 'X2', 'X3', 'X4', 'X5','X8',
                     'A0','A2',
                     'nextClick2','nextClick4','nextClick5',
                      'nextClick6','B1','B2','B3','B4',
                      'C0','C1'])
    categorical = ['app', 'device', 'os', 'channel', 'hour']
    print('predictors',predictors) # total 18+2+3+5+2=30 features
    
#     sub = pd.DataFrame()
#     sub['click_id'] = test_df['click_id']
#     print(id(sub['click_id']),id(test_df['click_id'])) # not same
    print('Generate 9th:4,5,9,10,13,14 +8h validation set...')
    val_df_real = val_df[(val_df.hour==12)|(val_df.hour==13)|(val_df.hour==17)|(val_df.hour==18)|(val_df.hour==21)|(val_df.hour==22)]
    val_df.drop(val_df_real.index, axis=0, inplace=True)
    gc.collect()
    train_df = pd.concat([train_df,val_df], copy=False) # axis : {0/’index’, 1/’columns’}, default 0
    
#     print('val_df_real:')
#     print(val_df_real.info())
#     print('Final train_df:')
#     print(train_df.info())
    del val_df
    gc.collect()
    
    test_df = pd.read_pickle('./test_df_V%d.pkl'%read_fileno) # 还未转换为float32
    test_df.drop(columns=['X6','ip_tchan_count','ip_app_channel_var_day','B0','A1'],inplace=True)
    testadd = pd.read_pickle('test_df_feature4.pkl')
    test_df['C0'] = testadd['C0'].values
    test_df['C1'] = testadd['C1'].values
    del testadd
    gc.collect()

    print("Training beings...")
    start_time = time.time()
    (bst,best_iteration) = lgb_modelfit_nocv(
                            train_df, 
                            val_df_real, 
                            predictors, 
                            target, 
                            early_stopping_rounds=80, 
                            verbose_eval=True, 
                            num_boost_round=3000, 
                            categorical_features=categorical)
    del train_df
    del val_df_real
    gc.collect()
    print('')
    print('[{}]: model training time'.format(time.time() - start_time))

    print("Predicting...")
    test_df[predictors] = test_df[predictors].astype('float32',copy=False)
    test_df['is_attributed'] = bst.predict(test_df[predictors],num_iteration=best_iteration)
    
    if not debug:
#         try:
        print("Writing result...")
        sub = pd.read_csv('../input/test.csv',dtype='uint32',usecols=['click_id'])
        test_df['click_id'] = sub
        test_df[['click_id','is_attributed']].to_csv(b'C:\Users\Administrator\Desktop\submit_V%d.csv'%(fileno),index=False,float_format='%.9f')
#         test_df[['click_id','is_attributed']].to_csv('./submit_V%d.csv'%(fileno),index=False,float_format='%.9f')
        print('Save feature importances...')
        lgb.plot_importance(bst)
        plt.gcf().savefig(b'C:\Users\Administrator\Desktop\V%d_%d_split.png'%(fileno,nchunk),bbox_inches='tight')
        lgb.plot_importance(bst,importance_type='gain')
        plt.gcf().savefig(b'C:\Users\Administrator\Desktop\V%d_%d_gain.png'%(fileno,nchunk),bbox_inches='tight')
#         except:
#             return test_df, bst
#     del test_df, bst
    gc.collect()
    
    print('The entire running time is:',time.time()-script_start_time)
    print("All finished...",time.asctime( time.localtime(time.time())))

Beginning... Mon May 14 08:59:57 2018
Reading dataset from disk file...
Loading data time: 911.7561986446381
predictors ['app', 'device', 'os', 'channel', 'hour', 'nextClick', 'ip_tcount', 'ip_app_count', 'ip_app_os_count', 'ip_app_channel_mean_hour', 'ip_app_os_var', 'X0', 'X1', 'X2', 'X3', 'X4', 'X5', 'X8', 'A0', 'A2', 'nextClick2', 'nextClick4', 'nextClick5', 'nextClick6', 'B1', 'B2', 'B3', 'B4', 'C0', 'C1']
Generate 9th:4,5,9,10,13,14 +8h validation set...
Training beings...
load train_df into lgb.Dataset...
load valid_df into lgb.Dataset...
Start doing lgb.train...


D:\anaconda3\lib\site-packages\lightgbm\basic.py:1036: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
D:\anaconda3\lib\site-packages\lightgbm\basic.py:681: UserWarning: categorical_feature in param dict is overrided.
  warnings.warn('categorical_feature in param dict is overrided.')


Training until validation scores don't improve for 80 rounds.
[10]	valid's auc: 0.962036
[20]	valid's auc: 0.963527
[30]	valid's auc: 0.967438
[40]	valid's auc: 0.96981
[50]	valid's auc: 0.971229
[60]	valid's auc: 0.972523
[70]	valid's auc: 0.973804
[80]	valid's auc: 0.975062
[90]	valid's auc: 0.976066
[100]	valid's auc: 0.976832
[110]	valid's auc: 0.977648
[120]	valid's auc: 0.978217
[130]	valid's auc: 0.978684
[140]	valid's auc: 0.979059
[150]	valid's auc: 0.979396
[160]	valid's auc: 0.979637
[170]	valid's auc: 0.979837
[180]	valid's auc: 0.980081
[190]	valid's auc: 0.980301
[200]	valid's auc: 0.98043
[210]	valid's auc: 0.980625
[220]	valid's auc: 0.980786
[230]	valid's auc: 0.980916
[240]	valid's auc: 0.980988
[250]	valid's auc: 0.981077
[260]	valid's auc: 0.981179
[270]	valid's auc: 0.981265
[280]	valid's auc: 0.981339
[290]	valid's auc: 0.98143
[300]	valid's auc: 0.981513
[310]	valid's auc: 0.981582
[320]	valid's auc: 0.98164
[330]	valid's auc: 0.9817
[340]	valid's auc: 0.981761
[

AttributeError: 'bytes' object has no attribute 'close'

In [2]:
dir(b'C:\Users\Administrator\Desktop')

['__add__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getnewargs__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__mod__',
 '__mul__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rmod__',
 '__rmul__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'capitalize',
 'center',
 'count',
 'decode',
 'endswith',
 'expandtabs',
 'find',
 'fromhex',
 'hex',
 'index',
 'isalnum',
 'isalpha',
 'isdigit',
 'islower',
 'isspace',
 'istitle',
 'isupper',
 'join',
 'ljust',
 'lower',
 'lstrip',
 'maketrans',
 'partition',
 'replace',
 'rfind',
 'rindex',
 'rjust',
 'rpartition',
 'rsplit',
 'rstrip',
 'split',
 'splitlines',
 'startswith',
 'strip',
 'swapcase',
 'title',
 'translate',
 'upper',
 'zfill']

In [2]:
# second training
dir() # If called without an argument, return the names in the current scope.

['DateOffset',
 'In',
 'Out',
 '_',
 '__',
 '___',
 '__builtin__',
 '__builtins__',
 '__doc__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_dh',
 '_i',
 '_i1',
 '_i2',
 '_ih',
 '_ii',
 '_iii',
 '_oh',
 '_sh',
 'best_iteration',
 'bst',
 'categorical',
 'debug',
 'do_count',
 'do_countuniq',
 'do_cumcount',
 'do_mean',
 'do_var',
 'exit',
 'fileno',
 'frm',
 'gc',
 'get_ipython',
 'lgb',
 'lgb_modelfit_nocv',
 'nchunk',
 'np',
 'nrows',
 'os',
 'pd',
 'plt',
 'predictors',
 'quit',
 'read',
 'read_fileno',
 'script_start_time',
 'start_time',
 'sub',
 'target',
 'test_df',
 'time',
 'to',
 'train_test_split',
 'val_size']

In [4]:
print("Writing result...")
sub = pd.read_csv('../input/test.csv',dtype='uint32',usecols=['click_id'])
test_df['click_id'] = sub
#     test_df[['click_id','is_attributed']].to_csv(b'C:\Users\Administrator\Desktop\submit_V%d.csv'%(fileno),index=False,float_format='%.9f')
test_df[['click_id','is_attributed']].to_csv('./submit_V%d.csv'%(fileno),index=False,float_format='%.9f')
print('Save feature importances...')
lgb.plot_importance(bst)
plt.gcf().savefig('./V%d_%d_split.png'%(fileno,nchunk),bbox_inches='tight')
lgb.plot_importance(bst,importance_type='gain')
plt.gcf().savefig('./V%d_%d_gain.png'%(fileno,nchunk),bbox_inches='tight')

Writing result...
Save feature importances...
